In [23]:
import pandas as pd
import numpy as np
import pickle
import json
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import os

In [24]:
with open('hashcode_df.pickle', 'rb') as handle:
    hashcode_df = pickle.load(handle)

In [25]:
def dhash(image, hash_size=8):
    pixels = prepare_image(image, hash_size, hash_size+1)
    diff = pixels[1:,:] > pixels[:-1,:]
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

def prepare_image(image, size1, size2=None):
    if not size2:
        size2 = size1
    res = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    res = cv2.resize(res, (size1, size2), interpolation=cv2.INTER_AREA)
    return res

def hamming(h1, h2):
    h, d = 0, h1 ^ h2
    while d:
        h += 1
        d &= d - 1
    return h

In [26]:
if __name__ == '__main__':
   
    with open('hashcode_df.pickle', 'rb') as handle:
        hashcode_df = pickle.load(handle)

    image_name = [str(x) for x in input().split()]
    directory = "/home/ankan/Projects/infilect/images"
    data = {}

    ## Calculating the similarity score of our input images with our inventory of tops images ##
    
    for file in image_name:
        image_path =  os.path.join(directory, file)
        image = cv2.imread(image_path)
        hash_code = dhash(image)
        sim_list = []
        name = file.split('.')[0]
        for index, row in hashcode_df.iterrows():
            dist = hamming(hash_code, row.hashcode)
            simm = (64 - dist) * 100 / 64
            if simm>=90 and row.file != image_name:
                sim_list.append((row.file.split('.')[0], simm))
            
        data[name] = sim_list
        
    ## Dumping the json output file ##    

    with open('near_duplicate_image.json', 'w') as outfile:
        json.dump(data, outfile)

TKTEE8A2XYSPYFWP.jpg TKTEC88BHKXH2PYZ.jpg
